<a href="https://colab.research.google.com/github/Gabriela2685/Simulacion2/blob/main/Un_servidor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ejemplo 5.2.1
Supongamos que las llegadas se producen según un NHPP co función de intensidad de(en horas) y con tiempo de cierre T_p = 9. Para el tiempo de atención en el servidor, en horas, se simula de una exponencial exp(20)

In [7]:
import math
import random

In [8]:
# Función para generar un tiempo aleatorio exponencial
def generar_tiempo_exponencial(tasa):
    return -math.log(1.0 - random.random()) / tasa

# Función lambda(t) para el NHPP
def lambda_t(t):
    if 0 <= t <= 3:
        return 5 + 5 * t
    elif 3 < t <= 5:
        return 20
    elif 5 < t <= 9:
        return 20 - 2 * (t - 5)
    else:
        return 0

# Parámetros de la simulación
T_p = 9  # Tiempo de cierre
lambda_max = 20  # Tasa máxima de llegadas
mu = 20  # Tasa de servicio
n_jornadas = 100  # Número de jornadas a simular

# Inicialización de variables para acumulación de métricas
acum_tiempo_medio_en_sistema = 0
acum_numero_medio_clientes = 0
acum_tiempo_despues_cierre = 0
acum_ultima_salida = 0

# Simulación para n jornadas
for _ in range(n_jornadas):
    t = 0
    n = 0
    N_LL = 0
    N_S = 0
    t_S = float('inf')

    LL = []  # Lista para tiempos de llegada
    S = []   # Lista para tiempos de salida

    # Generar llegadas utilizando el método de thinning
    while t < T_p:
        Tt = generar_tiempo_exponencial(lambda_max)
        t += Tt
        if t > T_p:
            break
        if random.random() <= lambda_t(t) / lambda_max:
            N_LL += 1
            n += 1
            LL.append(t)
            if n == 1:
                Y = generar_tiempo_exponencial(mu)
                t_S = t + Y

    # Procesar salidas
    for i in range(len(LL)):
        if LL[i] < t_S and LL[i] <= T_p:
            N_S += 1
            n -= 1
            S.append(t_S)
            if n > 0:
                Y = generar_tiempo_exponencial(mu)
                t_S = t_S + Y
            else:
                t_S = float('inf')
        elif t_S > T_p:
            Tp = max(0, t_S - T_p)
            break

    # Cálculo de métricas para la jornada actual
    if len(S) > 0:
        tiempo_total_en_sistema = sum([S[i] - LL[i] for i in range(len(S))])
        tiempo_medio_en_sistema = tiempo_total_en_sistema / N_S if N_S > 0 else 0
        numero_medio_clientes = tiempo_total_en_sistema / T_p
        ultima_salida = S[-1]
        tiempo_despues_cierre = ultima_salida - T_p if ultima_salida > T_p else 0
    else:
        tiempo_medio_en_sistema = 0
        numero_medio_clientes = 0
        ultima_salida = 0
        tiempo_despues_cierre = 0

    # Acumular las métricas para calcular promedios al final
    acum_tiempo_medio_en_sistema += tiempo_medio_en_sistema
    acum_numero_medio_clientes += numero_medio_clientes
    acum_tiempo_despues_cierre += tiempo_despues_cierre
    acum_ultima_salida += ultima_salida

# Cálculo de promedios sobre las n jornadas
promedio_tiempo_medio_en_sistema = acum_tiempo_medio_en_sistema / n_jornadas
promedio_numero_medio_clientes = acum_numero_medio_clientes / n_jornadas
promedio_tiempo_despues_cierre = acum_tiempo_despues_cierre / n_jornadas
promedio_ultima_salida = acum_ultima_salida / n_jornadas

# Resultados finales
print("Promedio del tiempo medio del cliente en el sistema:", promedio_tiempo_medio_en_sistema)
print("Promedio del número medio de clientes en el sistema:", promedio_numero_medio_clientes)
print("Promedio del tiempo después del cierre:", promedio_tiempo_despues_cierre)
print("Promedio de la última salida:", promedio_ultima_salida)

Promedio del tiempo medio del cliente en el sistema: 0.05220642789715902
Promedio del número medio de clientes en el sistema: 0.01698124649465377
Promedio del tiempo después del cierre: 0.0
Promedio de la última salida: 0.2702956232318406
